In [1]:
# https://plotly.com/python/mapbox-layers/
# https://plotly.com/python/discrete-color/

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import geopandas
import plotly.express as px
from shapely.geometry import Polygon, Point
from sodapy import Socrata
from d2i_tools import *
pd.options.mode.chained_assignment = None  # default='warn'

# read live status
def read_current_parkingSensors():
    cols = ["st_marker_id", "status", 
#             "disabilityext1","duration1","effectiveonph1",
#             "endtime1","exemption1","fromday1","starttime1","today1","typedesc1",
#             "description1", "description2","description3","description4","description5","description6",
            "bay_id"]
    
    apptoken = os.environ.get("SODAPY_APPTOKEN")
    domain = "data.melbourne.vic.gov.au"
    client = Socrata(domain, apptoken)

    r1 = client.get_all('vh2v-4nfs')  # read current parking sensors snapshot available
    r2 = client.get_all("ntht-5rk7")  # read parking restrictions records of all bays

    df1 = pd.DataFrame.from_dict(r1)
    df2 = pd.DataFrame.from_dict(r2)
    df = pd.merge(df1, df2, left_on='bay_id', right_on='bayid', how='left')
    df = df[cols].rename(columns={'st_marker_id':'marker_id'})
    return df

def create_fullUpdate(df):
    gdf = geopandas.read_file("datasets/onStreetParkingBays_baselist_bayCentroid.geojson")
    gdf["marker_id"] = gdf["marker_id"].str.upper()
    df["marker_id"] = df["marker_id"].str.upper()
    ngdf = gdf.merge(df, how="outer", on="marker_id").drop_duplicates()
    ngdf["status"] = ngdf["status"].fillna("unknown")
    ngdf = ngdf.drop(columns=["rd_seg_id","rd_seg_dsc"])
    print(f"{ngdf.shape[0]} parking sensors in full base list.")
    return ngdf

def plot_map(df,map_height=400,map_zoom=12, marker_max=9):
    fig = px.scatter_mapbox(df, lat="lati", lon="long", hover_name="marker_id", #hover_data=["description1","description2","description3","description4","description5","description6"],
                                color="status", size="area_m2", size_max=marker_max, zoom=map_zoom, height=map_height,
                                color_discrete_map={
                                    "unknown":"gray", "Unoccupied":"green", "Present":"red"},
                                category_orders={"status": ["unknown", "Unoccupied", "Present"]})
    # fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(mapbox_style="carto-positron")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [ ]:
dfrc = read_current_parkingSensors()
ngdf = create_fullUpdate(dfrc)
plot_map(ngdf,map_height=500,map_zoom=12, marker_max=9)
getCurrentPSstatus(dfrc)

In [ ]:
pin = (-37.812389676018476, 144.95817254056357)  # centre of Little Lonsdale St (btw William St & Queen St)
radius_m = 150  # radius in metres

gdf_filtered = geoCirClip(ngdf,pin[0],pin[1],radius_m)
plot_map(gdf_filtered,map_height=300,map_zoom=15, marker_max=9)

In [ ]:
# set polygon to geo-filter geometries that are within
lat1, lon1 = -37.81192120679105, 144.95628148775384
lat2, lon2 = -37.81036832849929, 144.9616049327078
lat3, lon3 = -37.806134720609734, 144.95972125218563
lat4, lon4 = -37.805700253666316, 144.95569649380283
lat5, lon5 = -37.80921289191341, 144.95508810009383
polygon = Polygon([(lon1, lat1), (lon2, lat2), (lon3, lat3), (lon4, lat4), (lon5, lat5)])

gdf_filtered2 = geoPolyClip(ngdf, polygon)
plot_map(gdf_filtered2,map_height=300,map_zoom=15, marker_max=10)

In [ ]:
# read bufferred / ETL'd time data
dfrh = pd.read_csv("datasets/003_vh2v-4nfs__bufferred.csv", parse_dates=["db_read_time"])
print(f"{dfrh.shape[0]} time records read")

# get the time data for the filtered list of parking sensors
dfrh_filtered2 = dfrh[dfrh["st_marker_id"].isin(gdf_filtered2["marker_id"])]

# bin specifies resampling frequency ("15min", "1H, "1D", "1W", "1M")
# bin_stat specifies resampled statistic for each resampled time interval

# hourly over 24 hours
startdt = "2021-09-01 11:00:00"
enddt = "2021-09-2 11:00:00"
dfts1 = timeIntStats(dfrh_filtered2, startdt, enddt, bin="1H", bin_stat="median")

dfts1.Present.plot(kind='bar',color='red', alpha=0.3)
dfts1.Unoccupied.plot(kind='bar',alpha=0.3)

In [ ]:
# line plot over many days in 15min intervals
startdt = "2021-08-27 21:00:00"
enddt = "2021-09-09 11:00:00"

dfts2 = timeIntStats(dfrh, startdt, enddt, bin="15min", bin_stat="median")

dfts2.Present.plot(kind='line',color='red', alpha=0.3)
dfts2.Unoccupied.plot(kind='line',alpha=0.3)

In [ ]:
# more interactive line plot with display of data when hover over data point
dfts2a = dfts2.reset_index(level=0)
px.scatter(dfts2a, x="time_interval", y=["Unoccupied","Present"], hover_name="time_interval")

In [ ]:
v# below plotly stacked bar chart is probably the best compared to above, and it's interactive
dfts2a = dfts2.reset_index(level=0)
px.bar(dfts2a, x="time_interval", y=["Unoccupied","Present"], hover_name="time_interval", 
       height=300, title="Number of parking bays",
       color_discrete_map={"unknown":"gray", "Unoccupied":"green", "Present":"red"},
       category_orders={"status": ["unknown", "Unoccupied", "Present"]})

In [ ]:
dfrh1 = dfrh[100000:150000]  # sample 50,000 records to animiate
dfrh1["st_marker_id"] = dfrh1["st_marker_id"].str.upper()
dfrh1 = dfrh1.rename(columns={"st_marker_id":"marker_id"})
ngdf1 = ngdf.drop(columns=["status"])
ndfrh = ngdf1.merge(dfrh1, how="right", on="marker_id")
print(ndfrh.shape)
fig = px.scatter_mapbox(ndfrh, lat="lati", lon="long", hover_name="marker_id",
                            animation_frame=ndfrh.db_read_time.astype(str), animation_group="status",
                            color="status", size="area_m2", size_max=9, zoom=12, height=500,
                            color_discrete_map={ 
                                "unknown":"gray", "Unoccupied":"green", "Present":"red"},
                            category_orders={"status": ["unknown", "Unoccupied", "Present"]})
# fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()